### Imported Libraries

In [1]:
import os
import math
import pickle
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join

import pymatgen as mg
from pymatgen import MPRester
from pymatgen.io.cif import CifParser
from pymatgen.io import ase
from pymatgen.transformations.standard_transformations import OxidationStateDecorationTransformation
from pymatgen.transformations.standard_transformations import AutoOxiStateDecorationTransformation
from pymatgen.symmetry.groups import *
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

from ase import atoms
from ase.io import read, write

from pprint import pprint
from tqdm import notebook as tqdm
from tqdm.auto import tqdm as tqdm_pandas
tqdm_pandas.pandas()
# import swifter

### API KEY for Materials Project

__To run the notebook, a Materials Project API key is required!__ API keys can be generated [here](https://materialsproject.org/open).

In [2]:
mpr = MPRester(api_key='j8SaTa1mrbQZUJ2j')

### 1a. Read in all Li-containing compounds from Materials Project
Operations come from mongodb. The syntax can be found [here](https://docs.mongodb.com/manual/reference/operator/query/).

Queryable parameters from Materials Project can be found [here](https://materialsproject.org/docs/api).

In [3]:
data = mpr.query(criteria={"elements": {"$in": ["Li"]}, "nelements": {"$gte": 2}}, 
               properties=["icsd_ids", "cif", "material_id", "pretty_formula", "spacegroup.symbol", "structure", "band_gap", "e_above_hull"])

  0%|          | 0/19472 [00:00<?, ?it/s]

The MPRester query returns a list of dictionaries, where each dictionary contains the properties of interest. Most of the data cleaning can be handled using the [Python Materials Genomics (pymatgen) library](https://pymatgen.org/). However, some operations are more conveniently accomplished via the [Atomic Simulation Environment (ASE)](https://wiki.fysik.dtu.dk/ase/index.html). We'll make a redundant list which contains the structures as [ase.atoms.Atoms](https://wiki.fysik.dtu.dk/ase/ase/atoms.html) type.  

In [4]:
AAA = ase.AseAtomsAdaptor
pymatgen_structures = []
ase_structures = []

for material in tqdm.tqdm(data):
    pymatgen_structures.append((material["structure"], material["icsd_ids"], material["material_id"], material["pretty_formula"], material["spacegroup.symbol"], material["band_gap"], material["e_above_hull"]))
    ase_structures.append(AAA.get_atoms(material["structure"])) 

  0%|          | 0/19472 [00:00<?, ?it/s]

### 1b. Read in all Li-containing compounds from ICSD
The ICSD Li-containing .cif files are contained in a seperate folder. Due to copyright, the cifs themselves are not shared in this data repository. The cifs have already been ordered using Pymatgen. Due to computational resources, about 10% of the cifs could not be ordered. 

In [5]:
path = os.path.join(os.getcwd(), 'Li Cifs sanitized3')

# keep track of any errors here
errors = []

# all files in the directory are .cifs
for filename in tqdm.tqdm(os.listdir(path)):
    filepath = os.path.join(path, filename)
    try: 
        # the ICSD tag is being grabbed with filename[:-4]
        pymatgen_structures.append((CifParser(filepath).get_structures()[0], [int(filename[:-4])]))
        ase_structures.append(read(filepath))
    except:
        errors.append(filepath)

  0%|          | 0/7905 [00:00<?, ?it/s]

/home/fall/anaconda3/envs/FALL2020/lib/python3.8/site-packages/pymatgen/io/cif.py:1123: UserWarning: Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: %s" % "\n".join(self.warnings))


Remove files with errors after manually checking them. 

In [6]:
for item in errors[1:]:
    os.remove(item)

Check that the pymatgen_structures and ase_structures lists contain the same number of structures. 

In [7]:
len(pymatgen_structures)==len(ase_structures)

True

### 1c. Find all dupliciate ICSD IDs and Merge

Some of the Materials Project entries point to ICSD IDs. To avoid having duplicate data, we'll find these IDs and merge them. The latest duplicate will become the master because this ensures that ICSD data (experimental) is conserved. 

In [9]:
# list for IDs that will be deleted
deletion_list = []
# list for mapping the ICSD_IDs from Materials Project to those from the ICSD
mapping_IDs = []

# Loop over all the structures in pymatgen_structures
for idx, val in enumerate(tqdm.tqdm(pymatgen_structures, desc='Finding duplicate ICSD_IDs')):
    # Check all the ICSD IDs for each structures - there can and often are multiple listed IDs
    for item in val[1]:
        # Loop over all structures in pymatgen_structures
        for edx, val2 in enumerate(pymatgen_structures):
            # if the ICSD ID is in the list
            if item in val2[1]:
                # if the ICSD ID is a later entry then mark the earlier entry for deletion
                if edx > idx:
                    # mark the earlier entry for deletion
                    deletion_list.append(idx)
                    
                    # preserve the associated Materials Project values in the mapping_IDs list
                    # these vals are just the locations for ICSD_ID, MP_ID, spacegroup, bandgap, and Ehull
                    mapping_IDs.append((val[1], val[2], val[4], val[5], val[6]))


print('There are {} entries marked for deletion.'.format(len(deletion_list)))

Finding duplicate ICSD_IDs:   0%|          | 0/27377 [00:00<?, ?it/s]

There are 3099 entries marked for deletion.


### 1d. Mark some additional boundary cases for deletion

Structures that contain deuterium are not of interest. Find and mark for deletion. 

In [10]:
deuterium_containing = 0
for idx, val in enumerate(tqdm.tqdm(pymatgen_structures, desc='Check for deuterium')):
    for element in val[0].symbol_set:
        if element == 'D':
            deletion_list.append(idx)
            deuterium_containing+=1
            
print('There are {} entries that contain deuterium.'.format(deuterium_containing))

Check for deuterium:   0%|          | 0/27377 [00:00<?, ?it/s]

There are 91 entries that contain deuterium.


Structures with over 500 distinct lattice positions would be very difficult to work with. Mark for deletion. 

In [11]:
greater_than_500 = 0
for idx, structure in enumerate(tqdm.tqdm(ase_structures, desc='Checking for structures with over 500 sites')):
    if structure.get_global_number_of_atoms() > 500:
        greater_than_500 += 1
        deletion_list.append(idx)

print("There are {} structures with >500 atoms".format(greater_than_500))

Checking for structures with over 500 sites:   0%|          | 0/27377 [00:00<?, ?it/s]

There are 20 structures with >500 atoms


Special case deletion: The structure $Li_7C_{120}$ (mp-1223102) throws errors. We'll just manually delete it. 

In [12]:
for idx, structure in enumerate(tqdm.tqdm(pymatgen_structures, desc='Checking for mp-1223102')):
    try:
        if structure[2] == 'mp-1223102':
            print("Structure mp-1223102 is at index {}.".format(idx))
            deletion_list.append(idx)
    except:
        pass

Checking for mp-1223102:   0%|          | 0/27377 [00:00<?, ?it/s]

Structure mp-1223102 is at index 237.


### 1e. Remove unwanted structures. Keep list of deleted structures for later. 

In [13]:
deleted_structures = [value for index, value in enumerate(pymatgen_structures) if index in deletion_list]
pymatgen_structures_sanitized = [value for index, value in enumerate(pymatgen_structures) if index not in deletion_list]
ase_structures_sanitized = [value for index, value in enumerate(ase_structures) if index not in deletion_list]

Check that the new lists are equivalent in length:

In [14]:
print("pymatgen_structures_sanitized and ase_structures_sanitized are equivalent length check:", len(pymatgen_structures_sanitized) == len(ase_structures_sanitized))
print("There are a total of {} structures.".format(len(pymatgen_structures_sanitized)))

pymatgen_structures_sanitized and ase_structures_sanitized are equivalent length check: True
There are a total of 25291 structures.


### 1f. Reformat the sanitized lists into a composite dataframe (structures_df)

In [15]:
structures_df = pd.DataFrame(pymatgen_structures_sanitized)
structures_df.columns = ['structure','ICSD_ID', 'MP_ID', 'pretty_formula', 'spacegroup', 'bandgap', 'e_hull']
structures_df['ase_structure'] = ase_structures_sanitized
structures_df['composition'] = None
structures_df['replacement'] = None

# grab the output of the formula function and store it in the 'composition' column
structures_df.composition = structures_df.structure.map(lambda x: x.formula)
structures_df.ICSD_ID = structures_df.ICSD_ID.map(lambda x: int(x[0]) if (len(x)>0) else 0)

In [16]:
structures_df.head(20)

,structure,ICSD_ID,MP_ID,pretty_formula,spacegroup,bandgap,e_hull,ase_structure,composition,replacement
0,"[[0. 0. 2.6255595] Li, [0. ...",180565,mp-1001790,LiO3,Imm2,0.0854,0.225420,"(Atom('Li', [0.0, 0.0, 2.6255594986], index=0)...",Li1 O3,None
1,"[[2.79072525 1.34013453 0.79373764] Li, [0.930...",188829,mp-1001825,LiBe,P2_1/m,0.0000,0.166972,"(Atom('Li', [2.79072525, 1.34013452726, 0.7937...",Li2 Be2,None
2,"[[2.412716 2.412716 2.412716] Li, [3.619074 3....",236959,mp-1001831,LiB,Fd-3m,1.4331,0.386054,"(Atom('Li', [2.412716, 2.412716, 2.412716], in...",Li2 B2,None
3,"[[0. 0. 0.] Li, [1.2797665 1.2797665 1.2797665...",184904,mp-1009009,LiF,Pm-3m,7.5195,0.287542,"(Atom('Li', [0.0, 0.0, 0.0], index=0), Atom('F...",Li1 F1,None
4,"[[1.4805505 1.9928345 2.448045 ] Li, [0. 0. 0....",180561,mp-1018789,LiO2,Pnnm,0.0000,0.084218,"(Atom('Li', [1.4805505, 1.9928345, 2.448045], ...",Li2 O4,None
5,"[[2.117169 1.22235078 2.00965125] Li, [ 2.11...",0,mp-1025406,LiP3,P6_3/mmc,0.0000,0.656358,"(Atom('Li', [2.117169, 1.22235077803, 2.009651...",Li2 P6,None
6,"[[ 2.387422 -1.37838176 3.43392642] Li, [2....",639879,mp-1079408,Li5In4,P-3m1,0.0000,0.000000,"(Atom('Li', [2.387422, -1.3783817567580003, 3....",Li5 In4,None
7,"[[4.8016225 0. 1.323284 ] Li, [2.655815...",0,mp-1094129,LiO2,R-3m,0.0733,0.572424,"(Atom('Li', [4.801622500000001, 0.0, 1.323284]...",Li1 O2,None
8,"[[ 2.736541 -1.57994549 2.491208 ] Li, [2....",0,mp-1094156,LiMg2,P-62m,0.0000,0.026255,"(Atom('Li', [2.736541, -1.5799454932180002, 2....",Li2 Mg4,None
9,"[[0. 0. 4.496241] Li, [2.119593 0...",0,mp-1094159,Li3Mg,I4/mmm,0.0000,0.003841,"(Atom('Li', [0.0, 0.0, 4.496241], index=0), At...",Li3 Mg1,None


Label ICSD data with corresponding MP_IDS from the deleted compounds (saved in mapping_IDs).

In [17]:
# count the number of mapping events
count = 0

# for each line in the mapping_IDs list
for entry in tqdm.tqdm(mapping_IDs, desc='Label ICSD structures with the corresponding MP ID'):
    # for each value in the ICSD_IDs column
    for idx, value in enumerate(structures_df['ICSD_ID']):
        # if the value exists in the list of ICSD_IDs from the mapping_IDs
        if value in entry[0]:
            # if the structure doesn't already have a Materials Project ID
            if structures_df.loc[idx, 'MP_ID'] is None:
                structures_df.at[idx, 'MP_ID'] = entry[1]
                structures_df.at[idx, 'spacegroup'] = entry[2]
                structures_df.at[idx, 'bandgap'] = entry[3]
                structures_df.at[idx, 'e_hull'] = entry[4]
                count+=1

print("Materials project IDs added for {} structures.".format(count))
                

Label ICSD structures with the corresponding MP ID:   0%|          | 0/3099 [00:00<?, ?it/s]

Materials project IDs added for 3034 structures.


Add the deleted structures back in as a possible replacment column.

In [18]:
# count the number of replacment events
count = 0

# for each value in the ICSD_IDs column
for idx, ICSD_value in enumerate(tqdm.tqdm(structures_df['ICSD_ID'], desc='Add deleted structures into a replacement column')):
    # for each line in the deleted structures list
    for idx_del, deleted_structure in enumerate(deleted_structures):
        # if the ICSD value corresponds to a value in the deleted structures ICSD list
        if ICSD_value in deleted_structure[1]:
            structures_df.at[idx, 'replacement'] = deleted_structure[0]
            count+=1
            
print("{} deleted structures were added back into the 'replacement' column.".format(count))

Add deleted structures into a replacement column:   0%|          | 0/25291 [00:00<?, ?it/s]

3040 deleted structures were added back into the 'replacement' column.


In [19]:
structures_df.head(20)

,structure,ICSD_ID,MP_ID,pretty_formula,spacegroup,bandgap,e_hull,ase_structure,composition,replacement
0,"[[0. 0. 2.6255595] Li, [0. ...",180565,mp-1001790,LiO3,Imm2,0.0854,0.225420,"(Atom('Li', [0.0, 0.0, 2.6255594986], index=0)...",Li1 O3,None
1,"[[2.79072525 1.34013453 0.79373764] Li, [0.930...",188829,mp-1001825,LiBe,P2_1/m,0.0000,0.166972,"(Atom('Li', [2.79072525, 1.34013452726, 0.7937...",Li2 Be2,None
2,"[[2.412716 2.412716 2.412716] Li, [3.619074 3....",236959,mp-1001831,LiB,Fd-3m,1.4331,0.386054,"(Atom('Li', [2.412716, 2.412716, 2.412716], in...",Li2 B2,None
3,"[[0. 0. 0.] Li, [1.2797665 1.2797665 1.2797665...",184904,mp-1009009,LiF,Pm-3m,7.5195,0.287542,"(Atom('Li', [0.0, 0.0, 0.0], index=0), Atom('F...",Li1 F1,None
4,"[[1.4805505 1.9928345 2.448045 ] Li, [0. 0. 0....",180561,mp-1018789,LiO2,Pnnm,0.0000,0.084218,"(Atom('Li', [1.4805505, 1.9928345, 2.448045], ...",Li2 O4,None
5,"[[2.117169 1.22235078 2.00965125] Li, [ 2.11...",0,mp-1025406,LiP3,P6_3/mmc,0.0000,0.656358,"(Atom('Li', [2.117169, 1.22235077803, 2.009651...",Li2 P6,None
6,"[[ 2.387422 -1.37838176 3.43392642] Li, [2....",639879,mp-1079408,Li5In4,P-3m1,0.0000,0.000000,"(Atom('Li', [2.387422, -1.3783817567580003, 3....",Li5 In4,None
7,"[[4.8016225 0. 1.323284 ] Li, [2.655815...",0,mp-1094129,LiO2,R-3m,0.0733,0.572424,"(Atom('Li', [4.801622500000001, 0.0, 1.323284]...",Li1 O2,None
8,"[[ 2.736541 -1.57994549 2.491208 ] Li, [2....",0,mp-1094156,LiMg2,P-62m,0.0000,0.026255,"(Atom('Li', [2.736541, -1.5799454932180002, 2....",Li2 Mg4,None
9,"[[0. 0. 4.496241] Li, [2.119593 0...",0,mp-1094159,Li3Mg,I4/mmm,0.0000,0.003841,"(Atom('Li', [0.0, 0.0, 4.496241], index=0), At...",Li3 Mg1,None


### 1g. Check to see if the structures are ordered and oxidation state decorated

In [20]:
unordered_count = 0
unordered = []
no_oxidation_count = 0
no_oxidation_state = []

for idx in tqdm.tqdm(np.arange(0, len(structures_df), 1), desc='Check for oxidation states and ordering'):
    if not structures_df.loc[idx, "structure"].is_ordered:
        unordered_count += 1
        unordered.append(idx)
    try:
        for site in structures_df.loc[idx, "structure"]:
            _ = site.specie.oxi_state
    except Exception as e:
        no_oxidation_count +=1
        no_oxidation_state.append(idx)
        
print("{} structures are disordered.".format(unordered_count))
print("{} structures are not charge decorated.".format(no_oxidation_count))

Check for oxidation states and ordering:   0%|          | 0/25291 [00:00<?, ?it/s]

98 structures are disordered.
17592 structures are not charge decorated.


### 1h. Apply charge decoration where possible. 

Pymatgen contains three general ways to charge decorate structures:
* Use the .add_oxdiation_state_by_guess method on a pymatgen structure. 
* Use the AutoOxiStateDecorationTransformation class.
* Create a dictionary of charges and apply based on atomic identity using the OxidationStateDecorationTransformation class.

In [21]:
# oxidation dictionary added to brute force oxidation states
oxidation_dictionary = {"H":1, "Li": 1, "Na":1, "K":1, "Rb": 1, "Cs":1, "Be":2, "Mg":2, "Ca":2, \
                        "Sr":2, "Ba":2, "Ra": 2, "B":3, "Al":3, "Ga":3, "In":3, "Tl":3, \
                        "C":4, "Si": 4, "Ge": 4, "Sn": 4, "Pb": 4, "N":-3, "P":5, "As":5, \
                        "Sb": 5, "Bi":5, "O":-2, "S":-2, "Se":-2, "Te":-2, "Po":-2, "F":-1, \
                       "Cl":-1, "Br":-1, "I":-1, "Sc":3, "Y":3, "Lu":3, "Ti":4, "Zr":4, "Hf":4, \
                       "V":5, "Nb":5, "Ta":5, "Cr":6, "Mo":4, "W":6, "Mn":7, "Tc":7, "Re":7, \
                       "Fe":3, "Ru":3, "Os":3, "Co": 3, "Rh":3, "Ir":3, "Cu":2, "Ag":1, "Au":3, \
                       "Zn":2, "Ni":2, "Cd":2, "Hg":2, "La":3, "Ce":3, "Pd":2, "Pm":3, "Ho":3, \
                        "Eu":3, "Np":3, "Pu":4, "Gd":3, "Sm":2, "Tb":3, "Tm":3, "Yb":3, "Ac":3, \
                       "Dy": 3, "Er":3, "Pr":3, "U":6, "Pt":2, "Nd":3, "Th":4, "Pa":5}

# two types of transformations taken from the pymatgen
oxidation_decorator = OxidationStateDecorationTransformation(oxidation_dictionary)
oxidation_auto_decorator = AutoOxiStateDecorationTransformation(distance_scale_factor=1)

In [22]:
def apply_charge_decoration(structure):
    """
    Function intended to be used with pandas.DataFrame.apply()

    For each structure passed into the function, it sequentially attempts three charge decoration strategies:
    (1) a manual charge decoration using the oxidation_dictionary
    (2) charge decoration using OxidationStateDecorationTransformation
    (3) charge decoration using the built-in add_oxidation_state_by_guess() method
    
    If any of these strategies result in a structure with nearly zero charge (<= 0.5), then the decoration is accepted.

    Parameters
    ----------
    structure : pymatgen.core.structure
        a pymatgen structure file

    Returns
    ------
    structure: pymatgen.core.structure
        either charge decorated or not (if the decorations failed)
    """
    
    # try the manual decoration strategy
    temp_structure = structure.copy()
    try:
        manually_transformed_structure = oxidation_decorator.apply_transformation(temp_structure)
        if abs(manually_transformed_structure.charge) < 0.5:
            return manually_transformed_structure
    except:
        pass
    
    # try Pymatgen's auto decorator
    temp_structure = structure.copy()
    try:
        auto_transformed_structure = oxidation_auto_decorator.apply_transformation(temp_structure)
        if abs(auto_transformed_structure.charge) < 0.5:
            return auto_transformed_structure
    except:
        pass
    
    # allow Pymatgen to guess the oxidation states
    temp_structure = structure.copy()
    try:
        structure.add_oxidation_state_by_guess()
        return structure
    except:
        pass 

    return structure

In [23]:
mask = [True if x in no_oxidation_state else False for x in structures_df.index]
structures_uncharged_df = structures_df[mask]
structures_df.loc[mask, 'structure'] = structures_uncharged_df['structure'].progress_apply(apply_charge_decoration)

  0%|          | 0/17592 [00:00<?, ?it/s]

In [24]:
unordered_count = 0
unordered = []
no_oxidation_count = 0
no_oxidation_state = []

for idx in tqdm.tqdm(np.arange(0, len(structures_df), 1), desc='Find the number of structures that remain disordered and/or charge undecorated.'):
    if not structures_df.loc[idx, "structure"].is_ordered:
        unordered_count += 1
        unordered.append(idx)
    try:
        for site in structures_df.loc[idx, "structure"]:
            _ = site.specie.oxi_state
    except Exception as e:
        no_oxidation_count +=1
        no_oxidation_state.append(idx)
        
print("{} structures are disordered.".format(unordered_count))
print("{} structures are not charge decorated.".format(no_oxidation_count))

  0%|          | 0/25291 [00:00<?, ?it/s]

98 structures are disordered.
98 structures are not charge decorated.


### 1i. Save the dataframe using pickle

In [25]:
save_path = os.path.join(os.getcwd(), 'structures_df_3p8_post_sanitize.pkl')
save_file = open(save_path, 'wb')
pickle.dump(structures_df, save_file)
save_file.close()

### 1j. Save deleted structures for future use

In [26]:
save_path_2 = os.path.join(os.getcwd(), 'structures_deleted_post_sanitize.pkl')
save_file = open(save_path_2, 'wb')
pickle.dump(deleted_structures, save_file)
save_file.close()

### 1k. Test to make sure data loads well

In [27]:
save_path = os.path.join(os.getcwd(), 'structures_df_3p8_post_sanitize.pkl')
open_file = open(save_path, 'rb')
structures_df = pickle.load(open_file)
open_file.close()

### 1l. Removal of structures that couldn't be charge decorated

In [28]:
unordered_count = 0
unordered = []
no_oxidation_count = 0
no_oxidation_state = []

for idx in tqdm.tqdm(np.arange(0, len(structures_df), 1), desc='Remove structures without charge decoration'):
    if not structures_df.loc[idx, "structure"].is_ordered:
        unordered_count += 1
        unordered.append(idx)
    try:
        for site in structures_df.loc[idx, "structure"]:
            _ = site.specie.oxi_state
    except Exception as e:
        no_oxidation_count +=1
        no_oxidation_state.append(idx)
        
print("{} structures are disordered.".format(unordered_count))
print("{} structures are not charge decorated.".format(no_oxidation_count))

for_removal = np.unique(unordered+no_oxidation_state)
structures_df = structures_df.drop(for_removal)
structures_df = structures_df.reset_index()

Remove structures without charge decoration:   0%|          | 0/25291 [00:00<?, ?it/s]

98 structures are disordered.
98 structures are not charge decorated.


In [29]:
structures_df.head(-50)

,index,structure,ICSD_ID,MP_ID,pretty_formula,spacegroup,bandgap,e_hull,ase_structure,composition,replacement
0,0,"[[0. 0. 2.6255595] Li0+, [0. ...",180565,mp-1001790,LiO3,Imm2,0.0854,0.225420,"(Atom('Li', [0.0, 0.0, 2.6255594986], index=0)...",Li1 O3,None
1,1,"[[2.79072525 1.34013453 0.79373764] Li0+, [0.9...",188829,mp-1001825,LiBe,P2_1/m,0.0000,0.166972,"(Atom('Li', [2.79072525, 1.34013452726, 0.7937...",Li2 Be2,None
2,2,"[[2.412716 2.412716 2.412716] Li0+, [3.619074 ...",236959,mp-1001831,LiB,Fd-3m,1.4331,0.386054,"(Atom('Li', [2.412716, 2.412716, 2.412716], in...",Li2 B2,None
3,3,"[[0. 0. 0.] Li+, [1.2797665 1.2797665 1.279766...",184904,mp-1009009,LiF,Pm-3m,7.5195,0.287542,"(Atom('Li', [0.0, 0.0, 0.0], index=0), Atom('F...",Li1 F1,None
4,4,"[[1.4805505 1.9928345 2.448045 ] Li0+, [0. 0. ...",180561,mp-1018789,LiO2,Pnnm,0.0000,0.084218,"(Atom('Li', [1.4805505, 1.9928345, 2.448045], ...",Li2 O4,None
...,...,...,...,...,...,...,...,...,...,...,...
25138,25236,[[4.4408921e-16 0.0000000e+00 3.6529499e+00] B...,155126,mp-5914,None,I4_1/amd,3.4434,0.003077,"(Atom('Li', [1.51319656, 1.51319656, 5.17415],...",Li12 B4 N8,"[[0. 0. 6.49296948] Li, [3.316..."
25139,25237,"[[-8.67783161 -7.32252453 -9.73148729] K+, [-2...",81336,None,None,None,NaN,NaN,"(Atom('K', [-2.068668159849999, 4.124882359346...",K6 Li3 Ta18 P9 O72,None
25140,25238,"[[0. 0. 0.] Cu2+, [4.66556584e-16 2.90125000e+...",290741,None,None,None,NaN,NaN,"(Atom('Li', [1.41455, 1.450625, 6.524324999999...",Li2 V2 Cu2 O8,None
25141,25239,"[[-4.73005 -2.73089564 -3.4418274 ] Li+, [-...",83279,None,None,None,NaN,NaN,"(Atom('Li', [-4.730049907308853e-08, 5.4617913...",Li1 Sm9 Si6 O26,None


### 1m. Routine to label ICSD structures with spacegroup

In [30]:
def spacegroup_labeler(structure):
    """
    Function intended to be used with pandas.DataFrame.apply()

    For each structure passed into the function, determine the space group.

    Parameters
    ----------
    structure : pymatgen.core.structure
        a pymatgen structure file

    Returns
    ------
    sg.get_space_group_symbol() : string
        the symbol for the spacegroup
    """
    
    try:
        s = mg.Structure.from_sites(structure)
        sg = SpacegroupAnalyzer(s)
        return sg.get_space_group_symbol()
    except:
        return None

missing_spacegroup_idx = structures_df[structures_df["spacegroup"].isna()].index.tolist()
print("Initial: {} rows are missing a spacegroup.".format(len(missing_spacegroup_idx)))

# make the dataframe to apply the spacegroup analyzer only to rows missing a spacegroup
mask = [True if x in missing_spacegroup_idx else False for x in structures_df.index]
structures_without_spacegroup_df = structures_df[mask]
structures_df.loc[mask, 'spacegroup'] = structures_without_spacegroup_df['structure'].progress_apply(spacegroup_labeler)


missing_spacegroup_idx = structures_df[structures_df["spacegroup"].isna()].index.tolist()
print("Post spacegroup labeling: {} rows are missing a spacegroup.".format(len(missing_spacegroup_idx)))

Initial: 4665 rows are missing a spacegroup.


  0%|          | 0/4665 [00:00<?, ?it/s]

Post spacegroup labeling: 6 rows are missing a spacegroup.


### 1n. Breakdown of spacegroups

In [31]:
spacegroup_counts = []
for i in np.arange(1, 231, 1):
    symbol = sg_symbol_from_int_number(i)
    print("{} - {}: {}".format(i, symbol, len(structures_df[structures_df["spacegroup"] == symbol])))
    spacegroup_counts.append(len(structures_df[structures_df["spacegroup"] == symbol]))

1 - P1: 4475
2 - P-1: 2290
3 - P121: 0
4 - P12_11: 0
5 - C121: 0
6 - P1m1: 0
7 - P1c1: 0
8 - C1m1: 0
9 - C1c1: 0
10 - P12/m1: 0
11 - P12_1/m1: 0
12 - C12/m1: 0
13 - P12/c1: 0
14 - P12_1/c1: 0
15 - C12/c1: 0
16 - P222: 3
17 - P222_1: 6
18 - P2_12_12: 18
19 - P2_12_121: 0
20 - C222_1: 60
21 - C222: 38
22 - F222: 11
23 - I222: 2
24 - I2_12_121: 0
25 - Pmm2: 70
26 - Pmc2_1: 72
27 - Pcc2: 0
28 - Pma2: 1
29 - Pca2_1: 83
30 - Pnc2: 11
31 - Pmn2_1: 104
32 - Pba2: 2
33 - Pna2_1: 376
34 - Pnn2: 13
35 - Cmm2: 31
36 - Cmc2_1: 100
37 - Ccc2: 18
38 - Amm2: 73
39 - Aem2: 3
40 - Ama2: 27
41 - Aea2: 20
42 - Fmm2: 6
43 - Fdd2: 46
44 - Imm2: 51
45 - Iba2: 0
46 - Ima2: 7
47 - Pmmm: 27
48 - Pnnn: 0
49 - Pccm: 4
50 - Pban: 0
51 - Pmma: 35
52 - Pnna: 22
53 - Pmna: 7
54 - Pcca: 5
55 - Pbam: 58
56 - Pccn: 14
57 - Pbcm: 27
58 - Pnnm: 62
59 - Pmmn: 68
60 - Pbcn: 119
61 - Pbca: 208
62 - Pnma: 702
63 - Cmcm: 201
64 - Cmce: 100
65 - Cmmm: 95
66 - Cccm: 3
67 - Cmme: 10
68 - Ccce: 1
69 - Fmmm: 3
70 - Fddd: 39
71 - Im

### 1o. Save a copy without ASE structures for 3.7 compatibility (comment out if not needed)

In [32]:
# savePath = os.path.join(os.getcwd(), 'structuresDF_3p8_postSanitize2_noASE.pkl')
# saveFile = open(savePath, 'wb')
# pickle.dump(newInstance.drop(['aseStructures'], axis=1), saveFile)
# saveFile.close()

### 1p. Create structure simplifications. Nine versions of the structure are used:

* __structure:__ The original structure. 
* __structure_A:__ Only the anions. All anions are modeled as S atoms. 
* __structure_AM:__ Only the anions and mobile atoms. Anions and mobil atoms are modeled as S and Li, respectively. 
* __structure_CAN:__ Only the cations, anions, and mobile atoms. Cations, anions, and mobile atoms are modeled as Al, S, and Li, respectively. 
* __structure_CAMN:__ All atoms are retained. Cations, anions, mobile atoms, and neutral atoms are modeled as Al, S, Li, and Mg, respectively. 
* __structure_A40:__ As with structure_A but the lattice volume is scaled to 40 cubic angstroms per anion. 
* __structure_AM40:__ As with structure_AM but the lattice volume is scaled to 40 cubic angstroms per anion. 
* __structure_CAN40:__ As with structure_CAN but the lattice volume is scaled to 40 cubic angstroms per anion. 
* __structure_CAMN40:__ As with structure_CAMN but the lattice volume is scaled to 40 cubic angstroms per anion. 

    

In [33]:
def structure_simplifications(structure_input, simplification_dict):
    """
    Function intended to be used with pandas.DataFrame.apply()

    The function takes a structure and then simplifies it based on the contents of simplification_dict.
    The simplified structure is returned. 


    Parameters
    ----------
    structure_input : pymatgen.core.structure
        a pymatgen structure file
        
    simplification_dict : dictionary
        A dictionary containing boolean values for the keys: 'C', 'A', 'M', 'N', '40'

    Returns
    ------
    structure: pymatgen.core.structure
        the simplified structure
    """
    
    # copy the structure in case modification fails
    structure = structure_input.copy()
    
    # create lists to keep track of the indices for the different atom types: cation, anion, mobile, neutral
    cation_list = []
    anion_list = []
    mobile_list = []
    neutral_list = []
    
    # create list to keep track of which atoms will be removed
    removal_list = []
    
    # integer to keep track of how to scale the lattice (for the representations that end in '40')
    scaling_counter = 0
    
    for idx, site in enumerate(structure):
        # grab the element name at the site
        element = site.species.elements[0].element.name
        # grab the charge at the site
        charge = site.specie.oxi_state
        
        # if the site is the mobile atom
        if element == 'Li':
            mobile_list.append(idx)
        else:
            # if the site holds a neutral atom
            if charge == 0:
                neutral_list.append(idx)
                scaling_counter+=1
                structure.replace(idx, mg.Specie("Mg", oxidation_state=charge))
            # if the site holds a cation
            elif charge>0:
                cation_list.append(idx)
                structure.replace(idx, mg.Specie("Al", oxidation_state=charge))
            # if the site holds an anion
            else:
                anion_list.append(idx)
                scaling_counter+=1
                structure.replace(idx, mg.Specie("S", oxidation_state=charge))
    
    # comparison to simplification_dict to decide which sites are removed
    if not simplification_dict['C']:
        removal_list += cation_list     
    if not simplification_dict['A']:
        removal_list += anion_list                
    if not simplification_dict['M']:
        removal_list += mobile_list
    if not simplification_dict['N']:
        removal_list += neutral_list
    
    # Special cases for the structures_A and structures_CAN representations
    # Some structures have only Li. For these we are going to handle them as anions (because every representations includes anions)
    if len(structure) == len(mobile_list):
        if not simplification_dict['M']:
            for idx in mobile_list:
                structure.replace(idx, mg.Specie("S", oxidation_state=charge))
    
    # Some structures have only neutrals or cations. For these we are going to handle them as anions (because every representation includes anions)
    elif len(structure) == len(removal_list):
        if len(neutral_list) > 0:
            for idx in neutral_list:
                structure.replace(idx, mg.Specie("S", oxidation_state=charge))
            structure.remove_sites(cation_list+mobile_list)
        elif len(mobile_list) > 0:
            for idx in mobile_list:
                structure.replace(idx, mg.Specie("S", oxidation_state=charge))
            structure.remove_sites(cation_list+neutral_list)
        elif len(cation_list) > 0:
            for idx in cation_list:
                structure.replace(idx, mg.Specie("S", oxidation_state=charge))
            structure.remove_sites(neutral_list+mobile_list)
    
    # otherwise just remove whatever is in the removal list
    else:
        structure.remove_sites(removal_list)
    
    # if simplification_dict indicates that the lattice should be scaled
    if simplification_dict['40']:              
        if scaling_counter > 0:
            structure.scale_lattice(40*scaling_counter)
    
    return structure

In [34]:
# the eight distinct simplification dictionary are hardcoded here
simplification_dict_A = {'C':False, 'A':True, 'M':False, 'N':False, '40':False}
simplification_dict_AM = {'C':False, 'A':True, 'M':True, 'N':False, '40':False}
simplification_dict_CAN = {'C':True, 'A':True, 'M':False, 'N':True, '40':False}
simplification_dict_CAMN = {'C':True, 'A':True, 'M':True, 'N':True, '40':False}
simplification_dict_A40 = {'C':False, 'A':True, 'M':False, 'N':False, '40':True}
simplification_dict_AM40 = {'C':False, 'A':True, 'M':True, 'N':False, '40':True}
simplification_dict_CAN40 = {'C':True, 'A':True, 'M':False, 'N':True, '40':True}
simplification_dict_CAMN40 = {'C':True, 'A':True, 'M':True, 'N':True, '40':True}

# application of the structure_simplification() function, applied once for each simplification dictionary
structures_df['structure_A'] = structures_df['structure'].progress_apply(structure_simplifications, simplification_dict=simplification_dict_A)
structures_df['structure_AM'] = structures_df['structure'].progress_apply(structure_simplifications, simplification_dict=simplification_dict_AM)
structures_df['structure_CAN'] = structures_df['structure'].progress_apply(structure_simplifications, simplification_dict=simplification_dict_CAN)
structures_df['structure_CAMN'] = structures_df['structure'].progress_apply(structure_simplifications, simplification_dict=simplification_dict_CAMN)
structures_df['structure_A40'] = structures_df['structure'].progress_apply(structure_simplifications, simplification_dict=simplification_dict_A40)
structures_df['structure_AM40'] = structures_df['structure'].progress_apply(structure_simplifications, simplification_dict=simplification_dict_AM40)
structures_df['structure_CAN40'] = structures_df['structure'].progress_apply(structure_simplifications, simplification_dict=simplification_dict_CAN40)
structures_df['structure_CAMN40'] = structures_df['structure'].progress_apply(structure_simplifications, simplification_dict=simplification_dict_CAMN40)

  0%|          | 0/25193 [00:00<?, ?it/s]

  0%|          | 0/25193 [00:00<?, ?it/s]

  0%|          | 0/25193 [00:00<?, ?it/s]

  0%|          | 0/25193 [00:00<?, ?it/s]

  0%|          | 0/25193 [00:00<?, ?it/s]

  0%|          | 0/25193 [00:00<?, ?it/s]

  0%|          | 0/25193 [00:00<?, ?it/s]

  0%|          | 0/25193 [00:00<?, ?it/s]

In [35]:
# inspection of the dataframe with the eight new simplifications
structures_df.head()

,index,structure,ICSD_ID,MP_ID,pretty_formula,spacegroup,bandgap,e_hull,ase_structure,composition,replacement,structure_A,structure_AM,structure_CAN,structure_CAMN,structure_A40,structure_AM40,structure_CAN40,structure_CAMN40
0,0,"[[0. 0. 2.6255595] Li0+, [0. ...",180565,mp-1001790,LiO3,Imm2,0.0854,0.225420,"(Atom('Li', [0.0, 0.0, 2.6255594986], index=0)...",Li1 O3,None,"[[0. 0. 5.35128645] S0+, [5.55...",[[0. 0. 2.6255595] Li0+],"[[0. 0. 5.35128645] Mg0+, [5.5...","[[0. 0. 2.6255595] Li0+, [0. ...","[[0. 0. 7.5441549] S0+, [-1.1102...",[[0. 0. 3.70147024] Li0+],"[[0. 0. 7.5441549] Mg0+, [-1.110...","[[0. 0. 3.70147024] Li0+, [0. ..."
1,1,"[[2.79072525 1.34013453 0.79373764] Li0+, [0.9...",188829,mp-1001825,LiBe,P2_1/m,0.0000,0.166972,"(Atom('Li', [2.79072525, 1.34013452726, 0.7937...",Li2 Be2,None,"[[2.79072525 1.17476284 3.09734748] S0+, [0.93...","[[2.79072525 1.34013453 0.79373764] Li0+, [0.9...","[[2.79072525 1.17476284 3.09734748] Mg0+, [0.9...","[[2.79072525 1.34013453 0.79373764] Li0+, [0.9...","[[3.39085782 1.42739017 3.76341775] S0+, [1.13...","[[3.39085782 1.62832427 0.96442725] Li0+, [1.1...","[[3.39085782 1.42739017 3.76341775] Mg0+, [1.1...","[[3.39085782 1.62832427 0.96442725] Li0+, [1.1..."
2,2,"[[2.412716 2.412716 2.412716] Li0+, [3.619074 ...",236959,mp-1001831,LiB,Fd-3m,1.4331,0.386054,"(Atom('Li', [2.412716, 2.412716, 2.412716], in...",Li2 B2,None,"[[1.206358 1.206358 1.206358] S0+, [0. 0. 0.] ...","[[2.412716 2.412716 2.412716] Li0+, [3.619074 ...","[[1.206358 1.206358 1.206358] Mg0+, [0. 0. 0.]...","[[2.412716 2.412716 2.412716] Li0+, [3.619074 ...","[[1.70997595 1.70997595 1.70997595] S0+, [0. 0...","[[3.41995189 3.41995189 3.41995189] Li0+, [5.1...","[[1.70997595 1.70997595 1.70997595] Mg0+, [0. ...","[[3.41995189 3.41995189 3.41995189] Li0+, [5.1..."
3,3,"[[0. 0. 0.] Li+, [1.2797665 1.2797665 1.279766...",184904,mp-1009009,LiF,Pm-3m,7.5195,0.287542,"(Atom('Li', [0.0, 0.0, 0.0], index=0), Atom('F...",Li1 F1,None,[[1.2797665 1.2797665 1.2797665] S-],"[[0. 0. 0.] Li+, [1.2797665 1.2797665 1.279766...",[[1.2797665 1.2797665 1.2797665] S-],"[[0. 0. 0.] Li+, [1.2797665 1.2797665 1.279766...",[[1.70997595 1.70997595 1.70997595] S-],"[[0. 0. 0.] Li+, [1.70997595 1.70997595 1.7099...",[[1.70997595 1.70997595 1.70997595] S-],"[[0. 0. 0.] Li+, [1.70997595 1.70997595 1.7099..."
4,4,"[[1.4805505 1.9928345 2.448045 ] Li0+, [0. 0. ...",180561,mp-1018789,LiO2,Pnnm,0.0000,0.084218,"(Atom('Li', [1.4805505, 1.9928345, 2.448045], ...",Li2 O4,None,"[[1.4805505 2.51837684 0.41921791] S0+, [1.48...","[[1.4805505 1.9928345 2.448045 ] Li0+, [0. 0. ...","[[1.4805505 2.51837684 0.41921791] Mg0+, [1.4...","[[1.4805505 1.9928345 2.448045 ] Li0+, [0. 0. ...","[[2.07903566 3.53638411 0.588679 ] S0+, [2.07...","[[2.07903566 2.798401 3.43762192] Li0+, [0. ...","[[2.07903566 3.53638411 0.588679 ] Mg0+, [2.0...","[[2.07903566 2.798401 3.43762192] Li0+, [0. ..."


### 1q. Save and reload dataframe

In [36]:
save_path = os.path.join(os.getcwd(), 'structures_df_3p8_post_sanitize_w_simplifications.pkl')
save_file = open(save_path, 'wb')
pickle.dump(structures_df, save_file)
save_file.close()

In [37]:
save_path = os.path.join(os.getcwd(), 'structures_df_3p8_post_sanitize_w_simplifications.pkl')
open_file = open(save_path, 'rb')
structures_df = pickle.load(open_file)
open_file.close()

### 1r. Use nglview to manually examine some of the simplifications.

In [38]:
import nglview as nv

In [39]:
structures_df.loc[1000, "structure"]

Structure Summary
Lattice
    abc : 5.273309274773669 8.331184752050396 8.892704483114459
 angles : 71.27636949794744 88.42532209537619 100.86183111618156
 volume : 361.74890480514074
      A : 5.270937 -0.069759 -0.141942
      B : -1.325141 4.36416 6.971854
      C : 0.35824 -5.53989 6.947048
PeriodicSite: Li+ (2.9726, 0.7833, 4.7070) [0.6670, 0.4695, 0.2200]
PeriodicSite: Li+ (-0.1466, 0.2302, 11.7376) [0.1670, 0.9695, 0.7201]
PeriodicSite: Li+ (1.3313, -2.0289, 9.0701) [0.3329, 0.5305, 0.7800]
PeriodicSite: Li+ (4.4504, -1.4759, 2.0396) [0.8330, 0.0305, 0.2800]
PeriodicSite: Li+ (3.9625, -2.1577, 10.4115) [0.8516, 0.6349, 0.8789]
PeriodicSite: Li+ (1.8103, -1.5349, 3.5232) [0.3516, 0.1349, 0.3789]
PeriodicSite: Li+ (0.3413, 0.9121, 3.3655) [0.1483, 0.3651, 0.1211]
PeriodicSite: Li+ (2.4936, 0.2892, 10.2540) [0.6484, 0.8651, 0.6211]
PeriodicSite: Li+ (3.5870, 2.9679, 6.3715) [0.8814, 0.8273, 0.1049]
PeriodicSite: Li+ (1.7930, -1.9492, 6.4300) [0.3813, 0.3273, 0.6049]
PeriodicSite: L

In [40]:
view = nv.show_pymatgen(structures_df.loc[1000, "structure"])
view._remote_call('setSize', args = ['', '600px'])
view.camera = 'orthographic'
view.add_unitcell()
view

NGLWidget()

In [41]:
view2 = nv.show_pymatgen(structures_df.loc[1000, "structure_CAMN40"])
view2._remote_call('setSize', args = ['', '600px'])
view2.camera = 'orthographic'
view2.add_unitcell()
view2

NGLWidget()